### In this notebook, we utilized the pre-trained layers of the VGG-face framework, excluding the final two layers, to extract facial descriptors from images. These descriptors serve as numerical representations of facial features. We then employed the cosine similarity function to quantify the similarity between these descriptors and the original images from each class in our dataset. This process allowed us to determine which class or identity is most similar to the input image, effectively classifying the input image based on its facial features...

In [1]:
from keras.models import *
from keras.layers import *
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow
from numpy import dot
from numpy.linalg import norm
import os
import re

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# VGG face model 
# Extracted from https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [4]:
# gdown https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5

In [5]:
model.load_weights('vgg_face_weights.h5')

In [9]:
import zipfile

# Specify the path to the zip file you want to unzip
zip_file_path = "aligned_mahdi_hamed.zip"

# Specify the directory where you want to extract the contents of the zip file
extracted_folder = "extracted_folder"

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of the zip file to the specified directory
    zip_ref.extractall(extracted_folder)

print(f"Successfully extracted contents to {extracted_folder}")

Successfully extracted contents to extracted_folder


In [32]:
# !unzip "FaceRecogntion/aligned_mahdi_hamed.zip"

In [10]:
vgg_face_descriptor = Model(inputs=model.layers[0].input
, outputs=model.layers[-2].output)

In [16]:
img = cv2.imread("extracted_folder/aligned_mahdi_hamed/hamed (1).jpg")
cv2.imshow("hamed", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
img = cv2.imread("extracted_folder/aligned_mahdi_hamed/hamed (1).jpg")
img = cv2.resize(img, (224,224))
img = np.expand_dims(img, axis=0)
description = vgg_face_descriptor.predict(img)

1/1 [==============================] - 6s 6s/step


In [25]:
def describe_img(imgname):
  path = ("extracted_folder/aligned_mahdi_hamed/"+imgname+".jpg")
  img = cv2.imread(path)
  img = cv2.resize(img, (224,224),)
  cv2.imshow("des", img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  img = np.expand_dims(img, axis=0)
  description = vgg_face_descriptor.predict(img)
  description = description[0]
  return description

In [26]:
 hamed1 = describe_img("hamed (1)")

1/1 [==============================] - 1s 556ms/step


In [27]:
mehdi1 = describe_img("mahdi (1)")

1/1 [==============================] - 1s 1s/step


In [30]:
def hamed_ya_mehdi(image):
  desc = describe_img(image)
  cos_sim_hamed = dot (desc , hamed1)/(norm(desc)*norm(hamed1))
  cos_sim_mehdi = dot (desc , mehdi1)/(norm(desc)*norm(mehdi1))
  if cos_sim_hamed >= 0.7:
    out = image+"  hamede"
  elif cos_sim_mehdi >= 0.7:
    out = image+"  mehdie"
  else:
    out = image+"na hamede na mehdiye"
  return out


In [31]:
directory = 'extracted_folder/aligned_mahdi_hamed/'
for image_path in os.listdir(directory):
  image_path = os.path.splitext(image_path)[0]
  print(hamed_ya_mehdi(image_path))

1/1 [==============================] - 1s 751ms/step
hamed (1)  hamede
1/1 [==============================] - 1s 824ms/step
hamed (2)  hamede
1/1 [==============================] - 1s 816ms/step
hamed (3)  hamede
1/1 [==============================] - 1s 706ms/step
hamed (4)  hamede
1/1 [==============================] - 1s 753ms/step
hamed (5)  hamede
1/1 [==============================] - 1s 745ms/step
hamed (6)  hamede
1/1 [==============================] - 1s 530ms/step
hamed (7)  hamede
1/1 [==============================] - 1s 569ms/step
mahdi (1)  mehdie
1/1 [==============================] - 1s 552ms/step
mahdi (2)  mehdie
1/1 [==============================] - 1s 528ms/step
mahdi (3)  mehdie
1/1 [==============================] - 0s 464ms/step
mahdi (4)  mehdie
1/1 [==============================] - 0s 471ms/step
mahdi (5)  mehdie
1/1 [==============================] - 1s 539ms/step
mahdi (6)  mehdie
1/1 [==============================] - 0s 490ms/step
mahdi (7)  mehdie
